### Regression Modeling Exercise
**Corey Solitaire**

**10/01/2020**

<div class="alert alert-block alert-success"></div>

# Imports:

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, explained_variance_score
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE

import warnings
warnings.filterwarnings("ignore")

import wrangle
import features

<div class="alert alert-block alert-success"></div>

# Exercises:

Using the data on **student grades** from this lesson, complete the following:

## 1. Split the data into train, validate, and test datasets.

## 2. Create a model that uses exam 1 to predict the final grade.

## 3. Create a model that uses exam 2 to predict the final grade.

## 4. Compare your models in the following manner:
    - Calculate the mean squared error
    - Visualize the residuals. Create a seperate visualization for each model.
    - Visualize the actual vs the predicted values. Create a seperate visualization for each model.
    - Bonus: Combine the seperate visualizations for each model into a single visualization. Is this visual helpful?

## 5. Create a model that uses exam 1 and exam 3 to predict final grade. How does this model compare to your previous ones?

## 6. Take your best preforming model and measure its performance on the validate data set. How does the performance differ between train and validate?

## 7. Make a 4th model with a slight difference like one more/less feature or a single hyperparameter that's different to see if you can beat that the last model's performance on validate.

## 8. Tune your models using validate to improve performance. Select the model w/ the best performance and evaluate that one on test, to get a more clear understanding of how it will perform on out-of-sample data.

## **Our scenario continues:**

As a customer analyst, I want to know who has spent the most money with us over their lifetime. I have monthly charges and tenure, so I think I will be able to use those two attributes as features to estimate total_charges. I need to do this within an average of $5.00 per customer. 

## 9. Run all your previous scripts that acquired, prepared, split, and scaled the telco churn data.

## 10. Fit 3 different linear models to your data, one with just tenure, one with just monthly_charges, and one with both.

## 11. Evaluate the models and your baseline.

## 12. Select the model that performed the best, and evaluate it with your validate data.

## 13. Make a 4th model with a slight difference like one more feature or a single hyperparameter that's different to see if you can beat that the last model's performance on validate.

## 14. Tune your models using validate to improve performance. Select the model w/ the best performance and evaluate that one on test, to get a more clear understanding of how it will perform on out-of-sample data.